# WEB SCRAPER

In [176]:
# Libraries

import bs4
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd

# Sources

Air Inequality
https://www.airlinequality.com/

In [18]:
# URL 
ae_url = 'https://www.airlinequality.com/'
ae_url_airfrance = 'https://www.airlinequality.com/airline-reviews/air-france/'

# First Test

In [20]:
# Open http connection 
#site= "http://en.wikipedia.org/wiki/StackOverflow"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(ae_url_airfrance,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page)
print(soup)

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7 lt-ie10" lang="en-GB"> <![endif]--><!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8 lt-ie10" lang="en-GB"> <![endif]--><!--[if IE 8]>    <html class="no-js lt-ie9 lt-ie10" lang="en-GB"> <![endif]--><!--[if IE 9]>    <html class="no-js lt-ie10" lang="en-GB"> <![endif]--><!--[if gt IE 8]><!--><html lang="en-GB"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<title>Air France Customer Reviews | SKYTRAX</title>
<!-- Google Chrome Frame for IE -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<!-- mobile meta -->
<meta content="True" name="HandheldFriendly"/>
<meta content="320" name="MobileOptimized"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<!-- icons & favicons -->
<link href="https://www.airlinequality.com/wp-content/themes/airlinequality2014new/library/images/apple-icon-touch.png" rel="apple-touch-icon"/>

In [23]:
type(soup)

bs4.BeautifulSoup

In [28]:
div = soup.find("div", {"class":"rating-10 rating-large"})
div

<div class="rating-10 rating-large">
<span itemprop="ratingValue">
													5</span>/<span itemprop="bestRating">10</span> </div>

In [29]:
span = div.find_all("span")
span

[<span itemprop="ratingValue">
 													5</span>, <span itemprop="bestRating">10</span>]

In [33]:
for tag in span:
    print(tag.text.strip())

5
10


In [36]:
page.close()

In [34]:
#cool

# Get Reviews

In [38]:
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(ae_url_airfrance,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page)

In [49]:
page_soup = BeautifulSoup(page, "html.parser")

In [47]:
l_titles = []

In [50]:
bodies = soup.body

In [52]:
bodies = soup.find_all("div", {"class":"body"})

In [197]:
l_titles = []
l_reviewer_data = []
l_verifications = []
l_reviews = []
l_stats = []
for i in range(len(bodies)):
    title = bodies[i].h2.text
    l_titles.append(title)

    reviewer_data = bodies[i].h3.text #to be proccessed
    l_reviewer_data.append(reviewer_data)

    review = bodies[i].find("div", {'class':'text_content'})
    verification = review.a.text
    l_verifications.append(verification)
    review = review.text #to be proccessed 
    l_reviews.append(review)

    l_review_value = []
    l_review_value_header = []
    l_review_value_stars = []

    stats = bodies[i].table
    tdTags = stats.find_all("td")
    #print(tdTags)
    for tag in tdTags:
        #print(tag['class'][0], tag)
        if 'header' in tag['class'][0]:
            l_review_value_header.append(tag.text)
        if tag['class'][0] == 'review-value':
            l_review_value.append(tag.text)
        if 'stars' in tag['class'][0]:
            tag_values = tag.find_all('span', {'class':'star fill'})
            l_review_value_stars.append(int(tag_values[-1].text))
    l_review_value[-1:-1] = l_review_value_stars
    l_stats.append(list(zip(l_review_value_header, l_review_value)))

print(len(l_titles) == len(bodies))
print(len(l_reviewer_data) == len(bodies))
print(len(l_verifications) == len(bodies))
print(len(l_reviews)== len(bodies))
print(len(l_stats) == len(bodies))

True
True
True
True
True


In [184]:
l_review_value_header

['Aircraft',
 'Type Of Traveller',
 'Seat Type',
 'Route',
 'Date Flown',
 'Seat Comfort',
 'Ground Service',
 'Value For Money',
 'Recommended']

In [199]:
l_df = list(zip(l_titles, l_reviewer_data, l_verifications, l_reviews))
column_names = ['title', 'reviewer_data', 'verified_trip', 'review' ]
df = pd.DataFrame(l_df, columns = column_names)

In [200]:
df

,title,reviewer_data,verified_trip,review
0,"""attentive, courteous service""",\n\nH Warlen (United States) 17th January 2020,Trip Verified,✅ Trip Verified | Paris to Chicago. Very atte...
1,"""not correspond to business class""",\n\nE Valkilev (Spain) 16th January 2020,Trip Verified,✅ Trip Verified | Paris to Madrid. The produc...
2,"""checked size and weight of cabin bags""",\n\nPavel Nemecek (Czech Republic) 9th January...,Trip Verified,✅ Trip Verified | Paris to Prague. First of a...
3,"""we wanted to change our route""",\n\nNadia Ferreira (South Africa) 6th January ...,Trip Verified,✅ Trip Verified | Johannesburg to Paris. Staf...
4,"""a terrible experience""",\n\nW Benson (Hong Kong) 4th January 2020,Trip Verified,✅ Trip Verified | Birmingham to Hong Kong via ...
5,"""impressed by the service""",\n\n2 reviews\n\n\n\nHugo Talbert (Canada) 1st...,Trip Verified,✅ Trip Verified | Montreal to Paris in July 2...
6,"""All four flights were good""",\n\nS Tegtmeier (Germany) 28th December 2019,Trip Verified,✅ Trip Verified | Munich to Lyon and Lyon to ...
7,"""the worst experience""",\n\nCraig Annandale (New Zealand) 28th Decembe...,Trip Verified,✅ Trip Verified | Paris to Milan. We upgraded ...
8,"""flights were okay as usual""",\n\nRoberto Barbato (United States) 28th Decem...,Trip Verified,✅ Trip Verified | Milan to Houston via Paris....
9,"""only allows 1 hand luggage""",\n\nM Karune (United States) 26th December 2019,Trip Verified,✅ Trip Verified | Paris to Nairobi. We arrive...


In [203]:
df_stats = pd.DataFrame(l_stats[0])

In [205]:
l_stats

[[('Aircraft', 'A330-300'),
  ('Type Of Traveller', 'Business'),
  ('Seat Type', 'Business Class'),
  ('Route', 'Paris to Chicago'),
  ('Date Flown', 'January 2020'),
  ('Seat Comfort', 4),
  ('Cabin Staff Service', 4),
  ('Food & Beverages', 4),
  ('Inflight Entertainment', 4),
  ('Ground Service', 2),
  ('Wifi & Connectivity', 3),
  ('Value For Money', 4),
  ('Recommended', 'yes')],
 [('Aircraft', 'A320'),
  ('Type Of Traveller', 'Solo Leisure'),
  ('Seat Type', 'Business Class'),
  ('Route', 'Paris to Madrid'),
  ('Date Flown', 'January 2020'),
  ('Seat Comfort', 1),
  ('Cabin Staff Service', 2),
  ('Food & Beverages', 1),
  ('Inflight Entertainment', 1),
  ('Ground Service', 3),
  ('Wifi & Connectivity', 1),
  ('Value For Money', 1),
  ('Recommended', 'no')],
 [('Aircraft', 'A321'),
  ('Type Of Traveller', 'Solo Leisure'),
  ('Seat Type', 'Economy Class'),
  ('Route', 'Paris to Prague'),
  ('Date Flown', 'December 2019'),
  ('Seat Comfort', 3),
  ('Cabin Staff Service', 4),
  ('Foo

In [185]:
df['stats'].apply(pd.Series)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,A330-300,Business,Business Class,Paris to Chicago,January 2020,4,4,4,4,2.0,3.0,4,yes
1,A320,Solo Leisure,Business Class,Paris to Madrid,January 2020,1,2,1,1,3.0,1.0,1,no
2,A321,Solo Leisure,Economy Class,Paris to Prague,December 2019,3,4,4,1,5.0,3.0,yes,NaN
3,Family Leisure,Business Class,Johannesburg to Paris,January 2020,4,1,1,3,1,3.0,1.0,no,NaN
4,Boeing 777,Solo Leisure,Economy Class,Birmingham to Hong Kong via Paris,January 2020,1,2,2,2,1.0,1.0,1,no
5,Boeing 777-300ER,Solo Leisure,Economy Class,Montreal to Paris,July 2019,5,5,5,4,5.0,4.0,yes,NaN
6,A320 / A319 / E190,Solo Leisure,Economy Class,Munich to Lyon via Paris,October 2019,4,5,5,3,5.0,4.0,yes,NaN
7,Business,Economy Class,Paris to Milan,December 2019,1,1,1,1,1,1.0,1.0,no,NaN
8,Couple Leisure,Economy Class,Milan to Houston via Paris,October 2019,4,4,4,2,yes,NaN,NaN,NaN,NaN
9,A330,Couple Leisure,Economy Class,Paris to Nairobi,December 2019,1,1,1,no,NaN,NaN,NaN,NaN


In [179]:
l_df

[('"attentive, courteous service"',
  '\n\nH Warlen (United States) 17th January 2020',
  'Trip Verified',
  '✅ Trip Verified |  Paris to Chicago. Very attentive, courteous service in flight by the cabin crew on AF 0136. I wish our US domestic carriers could follow suit I highly recommend Air France.',
  ['A330-300',
   'Business',
   'Business Class',
   'Paris to Chicago',
   'January 2020',
   4,
   4,
   4,
   4,
   2,
   3,
   4,
   'yes']),
 ('"not correspond to business class"',
  '\n\nE Valkilev (Spain) 16th January 2020',
  'Trip Verified',
  '✅ Trip Verified |  Paris to Madrid. The product does not correspond to business class. The seats are the same as in the economy, the distance for the legs is not increased, the middle chair does not even transform into a table. The food is scarce and tasteless - two cold starters, a slice of cheese and a little chocolate bar. The staff responds to requests, but they are not particularly willing to do so. Wines in small bottles on a screw

In [148]:
l_review_value[-2:-1] = l_review_value_stars

In [149]:
l_review_value

['A330-300',
 'Business',
 'Business Class',
 'Paris to Chicago',
 4,
 4,
 4,
 4,
 2,
 3,
 4,
 'yes']

TripAdvisor https://www.tripadvisor.com/Airlines

In [18]:
# URL 

ta_url = 'https://www.tripadvisor.com/Airline_Review-d8729003-Reviews-Air-France'

In [19]:
# Open http connection 

uClient = uReq(ta_url)

In [20]:
# Download web page html content

page_html = uClient.read()

In [21]:
# Close http connection

uClient.close()

In [22]:
# Parse the web page to get bs4 data structure

page_soup = soup(page_html, "html.parser")

In [38]:
# Get Header of the page

page_soup.h1

<h1 class="header heading masthead masthead_h1">Air France Reviews and Flights</h1>

In [41]:
# Get reviews

reviews = page_soup.findAll("div", 
                  {"class":'location-review-card-Card__ui_card--2Mri0.location-review-card-Card__card--o3LVm.location-review-card-Card__section--NiAcw'})



In [45]:
len(reviews)

0

SkyTrax https://skytraxratings.com/airlines

Twitter's airline hashtag feed

In [3]:
import tweepy
import csv
import pandas as pd
####input your credentials here
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
#####Air France
# Open/Create a file to append data
csvFile = open('airfrancetweet.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,q="#airfrance",count=100,
                           lang="en",
                           since="2017-04-03").items():
    print (tweet.created_at, tweet.text)
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

ModuleNotFoundError: No module named 'tweepy'